Using a programming language that you are familiar with, such as C++ or Java, implement three frequent itemset mining algorithms
introduced in this chapter: (1) Apriori [AS94b], (2) FP-growth [HPY00]. (mining using the vertical data format). Compare the performance of each algorithm with various kinds of large data sets. Write a report to
analyze the situations (e.g., data size, data distribution, minimal support threshold
setting, and pattern density) where one algorithm may perform better than the
others, and state why.

Reference: https://rdrr.io/cran/arules/man/Adult.html

In [24]:
import datetime as dt
import pandas as pd
import numpy as np
import random as rand
import csv
import time
import copy
from itertools import combinations


In [25]:
#dataset test
file = r'E:\School\@Grad School\Data Mining\Project 1\adult.data'
#file.read()

df = pd.read_csv(file, sep=',',
                 names=["Age", "Workclass", "fnlwgt","education", "education_num",
                        "marital status","occupation","relationship","Race","Sex",
                        "capital_gain","capital_loss","hours_per_week","native country",
                        "income"],
                 skipinitialspace=True)
df.head()



,Age,Workclass,fnlwgt,education,education_num,marital status,occupation,relationship,Race,Sex,capital_gain,capital_loss,hours_per_week,native country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [26]:
def clean_data(df):
    df= df.applymap(lambda x: np.nan if x == '?' else x)
    df=df.dropna(axis=0)
    df.drop('fnlwgt', axis=1, inplace=True)
    df.drop('education_num', axis=1, inplace=True)
    df['Age'] = pd.cut(df['Age'], [0, 26, 46, 66,100], 
                        labels = ['Young', 'Middle-aged', 'Senior', 'Old'], 
                        right = True, include_lowest = True)
    df['hours_per_week'] = pd.cut(df['hours_per_week'], [0, 25, 40, 60, 100], 
                              labels = ['Part-time', 'Full-time', 'Over-time', 'Too-much'], 
                              right = True, include_lowest = True)
    df['capital_gain'] = pd.cut(df['capital_gain'], [0, 1, 10000000], 
                           labels = ['No-Gain', 'Positive-Gain'], 
                           right = True, include_lowest = True)
    df['capital_loss'] = pd.cut(df['capital_loss'], [0, 1, 10000000],
                            labels = ['No-Loss', 'Positive-Loss'], 
                            right = True, include_lowest = True)


    return df



In [27]:
data = [['M', 'O', 'N', 'K', 'E', 'Y'], 
            ['D', 'O', 'N', 'K', 'E', 'Y'], 
            ['M', 'A', 'K', 'E'],
            ['M', 'U', 'C', 'K', 'Y'],
            ['C', 'O', 'O', 'K', 'I', 'E']]

def open_data():
    
    
    data = clean_data(df)
    dataset= data.values.tolist()
    return dataset
dataset= open_data()
dataset;

In [28]:
#obtain C1 with support count


def obtain_C1(dataset):
    C1 = {}
    for item in dataset:
        for itemset in item:
            if itemset in C1:
                C1[itemset] += 1
            elif itemset not in C1:
                C1[itemset] = 1
    return C1
            
            
C1=obtain_C1(open_data())
C1;

In [29]:
#implement 1-itemset
def gen_L1 (min_sup):
    L1 = []
    num_items = float(len(dataset))
    #print(num_items)
    for item in C1:
        support = C1[item]/num_items
        if support >= min_sup:
            L1.append(item)
    return L1
            
L1=gen_L1(0.6)
L1

['White', 'Male', 'No-Loss', 'United-States', '<=50K', 'No-Gain', 'Private']

In [30]:
# c: candidate k-itemset, L: frequent (k-1)-itemsets, k: # of c
def has_infrequent_subset(c, L):
    k= len(c)
    for subset in list(combinations(c,k-1)):
        if subset not in L:
            return True
    return False


#generate Ck
def apriori_gen(L, k):  
    Ck = list(combinations(L, k))
    #Ck = []
    lenL = len(L)
    
    return Ck

apriori_gen(L1,2);

In [31]:
#find Lk
def prune (dt, Ck, min_sup):
    count={}
        
    for item in dt:
        for itemset in Ck:
            if set(itemset).issubset(item):
                if itemset in count:
                    count[itemset] += 1
                else:
                    count[itemset] = 1
    #generate Lk 
    num_items = float(len(dt))
    Lk = []
    #copy_count = copy.deepcopy(count)
    for itemset in count:
        support = count[itemset]/num_items
        if support >= min_sup:
            Lk.append(itemset)
            
            
    return Lk


In [32]:
def apriori(dataset, min_sup=0.8):
    C1 = obtain_C1(dataset)
    D= dataset
    L1= gen_L1(min_sup)
    #Lk=[]
    #count={}
    L =[]
    L.append(L1)
    k=2
    while len(L[k-2]) >0:    
        Ck =  apriori_gen(L1, k)
        Lk = prune(dataset, Ck, min_sup)

        L.append(Lk)
        k+=1
    #return L, support_data
    return L[0:len(L)-1]
                

In [33]:
start = time.time()
apriori(dataset,0.8)
run = time.time() -start
run

0.37223243713378906

In [34]:
apriori(dataset)

[['White', 'No-Loss', 'United-States', 'No-Gain'],
 [('White', 'No-Loss'),
  ('White', 'United-States'),
  ('No-Loss', 'United-States'),
  ('No-Loss', 'No-Gain'),
  ('United-States', 'No-Gain')]]